## Vanilla LSTM
### Based mostly on code and tutorial at

(Someday I only hope I'll be able to write posts as well as the below)
- https://github.com/suriyadeepan/rnn-from-scratch/blob/master/vanilla.py

- http://suriyadeepan.github.io/2017-02-13-unfolding-rnn-2/

Some other resources to understand, especially, the origin of LSTMs and word-embeddings
- https://r2rt.com/written-memories-understanding-deriving-and-extending-the-lstm.html
- http://sebastianruder.com/word-embeddings-1/

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
state_size = 37 # (parameter for the RNN cell; describes how wide the cell should be)
embedding_size = 18 #(parameter for embedding layer; describes the size of embedding for each token)

### Mutable variables for weights and biases
- W: weight matrices  for the recurrent relationship between cell states
- U: weight matrices  for the connection between input and cell state
- b: bias parameter for input -> cell state connections

In [3]:
W_i = tf.Variable(initial_value=tf.random_normal(mean=0.,stddev=0.1,shape=[state_size, state_size]))
W_f = tf.Variable(initial_value=tf.random_normal(mean=0.,stddev=0.1,shape=[state_size, state_size]))
W_o = tf.Variable(initial_value=tf.random_normal(mean=0.,stddev=0.1,shape=[state_size, state_size]))
W_g = tf.Variable(initial_value=tf.random_normal(mean=0.,stddev=0.1,shape=[state_size, state_size]))

In [4]:
U_i = tf.Variable(initial_value=tf.random_normal(mean=0.,stddev=0.1,shape=[embedding_size, state_size]))
U_f = tf.Variable(initial_value=tf.random_normal(mean=0.,stddev=0.1,shape=[embedding_size, state_size]))
U_o = tf.Variable(initial_value=tf.random_normal(mean=0.,stddev=0.1,shape=[embedding_size, state_size]))
U_g = tf.Variable(initial_value=tf.random_normal(mean=0.,stddev=0.1,shape=[embedding_size, state_size]))

### Placeholder for input and target

In [5]:
ph_xs = tf.placeholder(shape=[None, None], dtype=tf.int32)
ph_ys = tf.placeholder(shape=[None, None], dtype=tf.int32)

In [6]:
ph_init_state = tf.placeholder(shape=[None, state_size], dtype=tf.float32, name="initial_state")

### Data Acquisition
The data in the files were populated using the *Data_Provider* notebook. *train_x* and *train_y* are both arrays of shape: [10,200], where 200 is a fixed sequence length for this model and 10 is the total number of samples we have available. 

In [7]:
train_x = np.load("train_x.npy")
train_y = np.load("train_y.npy")

def get_train_batches(train_x, train_y, batch_size):
    for i in range(0, train_x.shape[0], batch_size):    
        yield train_x[i : i+batch_size], train_y[i : i+batch_size]

### Embedding
Given an input data of shape: [batch_size, seq_length], rnn_inputs returns a tensor of shape: [batch_size, seq_length, embedding_size]. This is purely driven by our decision to use an embedding matrix to represent each token (characters). Otherwise, we would've had to one-hot encode each character, and modify our input layer. I'm not sure what the complexity of this task would need to be, but using embedding matrices seem to be the preferred approach in most NLP applications of neural network. I think it massively improves the underlying computational complexity as well.

In [8]:
# number of unique characters.. you'd normally do this by inspecting the data directly
num_classes = 49 

In [9]:
embeddings = tf.Variable(initial_value=tf.random_normal(mean=0., stddev=0.1, shape=[num_classes,embedding_size]))

In [10]:
rnn_inputs = tf.nn.embedding_lookup(embeddings, ph_xs)

In [11]:
print("rnn_inputs: ",rnn_inputs.shape.as_list())

rnn_inputs:  [None, None, 18]


### The main equation that powers LSTMs

The equation below describes how cell-state at current time ($s_{t}$) must be updated as a function of the cell-state in the previous step ($s_{t-1}$), and input word at the current step ($x_t$). Note that the driving factors for the update are the factors: $i$, $f$, $o$, and most importantly, a hidden internal state $c_t$ in addition to the exposed internal weights $s_t$. The latter is considered exposed because it is also used to compute the outgoing logits, and hence the error vectors.

#### [update the input (i), forget(f), and output(o) gates factors]

$$ i = \sigma ( x_t U^i + s_{t-1} W^i ) $$
$$ f = \sigma ( x_t U^f + s_{t-1} W^f ) $$
$$ o = \sigma ( x_t U^o + s_{t-1} W^o ) $$

#### [update the hidden internal state: $c_t$ and the exposed internal state: $s_t$ 

$$ g = tanh ( x_t U^g + s_{t-1} W^g ) $$
$$ c_t = c_{t-1} \otimes f + g \otimes i $$
$$ s_t = tanh(c_t) \otimes o $$

Note that we're excluding the bias term for simplicity. Also the operator $\otimes$ denotes an element-wise multiplication of the vectors.

In [12]:
def step(previous, x):
    '''
    Method evaluates the new cell-state based on hidden
    states from the previous step and input for given step.

    
    parameters:
    - previous: This is the computed states from the
    previous time_step. See the unrolled computational
    graph in any vanilla LSTM tutorial.
    
    - x: input for current time_step    
    '''
    st_1, ct_1 = tf.unstack(previous)
    
    # Equations for the LSTM
    i = tf.sigmoid(tf.matmul(x, U_i) + tf.matmul(st_1, W_i))
    f = tf.sigmoid(tf.matmul(x, U_f) + tf.matmul(st_1, W_f))
    o = tf.sigmoid(tf.matmul(x, U_o) + tf.matmul(st_1, W_o))
    g = tf.sigmoid(tf.matmul(x, U_g) + tf.matmul(st_1, W_g))    
    
    ct = ct_1*f + g*i
    st = tf.tanh(ct)*o
    
    return tf.stack([st,ct])

### tf.scan

Method builds a loop that dynamically unforlds, to recursively apply the function step over all elements of the rnn_inputs. This is also one of the most important step to intuitively understand. I think without using the step function, it'd be extremely complex to implement the recursion successfully (and/or accurately)

The dimensions also need to be reshuffled, so that the sequence length dimension is exposed as the 0th dimension. This is to enable iteration over elements of the sequence. I.e. Tensor of form [batch_size, seq_length, embedding_size] is transposed to [seq_length, batch_size, embedding_size]. The reshuffled tensor is represented by *transposed_inputs*

The _states_ returned by scan is an array of states from all the time steps, using which we will predict the output probabilities at each step.

In [13]:
transposed_inputs = tf.transpose(rnn_inputs,[1,0,2])

'''
tf.scan applies the step method recursively on the second argument, i.e
transposed_inputs. The returned value from one execution will also be the input 
to the next call of the method. 
'''
states = tf.scan(step, # the method that should be applied to each vec from below. Updates weights
                 transposed_inputs, # [vec1, vec2, .... , vecN]: each vec. corresponds to the embedding row for a word.
                                    # There are seqlen num. of words for every single batch. Hence, this method will recurse throug
                                    # seqlen * batch_size times, each time returning a vector of length: state_size
                 initializer=tf.stack([ph_init_state, ph_init_state])) # initializer. For computing the first argument (i.e. vec1), begin by a random set of values.

In [15]:
# [seq_len, state_type, batch_size, state_size]
# where: state_type = s_t and c_t where the former (see 'step' function)
# s_t: exposed state that will be used to evaluate logits in relation with
#      the output layer, and
# c_t: internal state that is used solely for updating LSTM cell's weights

print("states: ",states.shape.as_list())

states:  [None, 2, None, 37]


### Output

The *states* tensor returned by _scan_ is of shape: [seqlen * batch-size, state_size]. Thus reshape to allow vector multiplication against the output network involving V and bo, where:

- V: weight matrices for connecting cell-state to the output (or target)
- b: bias vector for connecting cell-state to the output

In [16]:
# Define weights for the output layer
V = tf.Variable(initial_value= tf.random_normal(mean=0., stddev=0.1, shape=[state_size, num_classes]))
bo = tf.Variable(initial_value= tf.random_normal(mean=0., stddev=0.1, shape=[num_classes]))

In [17]:
# The consequence of this is permuting the dimensions of states so we get:
# [state_type, batch_size, seqlen, state_size]
states = tf.transpose(states, [1,2,0,3])

# And get the first state_type (i.e. only s_t) because we only
# need that to evaluate that to get the logits
states = states[0]

In [18]:
# Now finally reshape to get [seqlen * batch_size, state_size]
states_reshaped = tf.reshape(states, [-1, state_size])

In [19]:
logits = tf.matmul(states_reshaped, V) + bo
predictions = tf.nn.softmax(logits)

### Optimization
ph_ys is shaped as follows: [batch_size, seq_length]. Look here to see what is happening in cell below. https://github.com/apiltamang/tensorflow_rtp_materials/blob/master/week-7/Vanilla_RNN_Using_Embeddings.ipynb

In [20]:
ph_ys_reshaped = tf.reshape(ph_ys, shape=[-1])

losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=ph_ys_reshaped, logits=logits)

loss = tf.reduce_mean(losses)

train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

### Training

In [21]:
batch_size = 2 # Take 2 at a time for the given 10 total samples we have available.
seq_length = 200 # each vector is fixed length of 200

In [23]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_loss = 0.
    n_epochs = 3
    for i in range(n_epochs):
        for xs, ys in get_train_batches(train_x, train_y, batch_size=2):
            _, train_loss_val = sess.run([train_op, loss], 
                                         feed_dict={ph_xs: xs, 
                                                    ph_ys: ys,
                                                    ph_init_state: np.zeros([batch_size, state_size])})
            print("loss evaluated: ",train_loss_val)

loss evaluated:  3.89412
loss evaluated:  3.75693
loss evaluated:  3.68188
loss evaluated:  3.58471
loss evaluated:  3.42063
loss evaluated:  3.25519
loss evaluated:  3.08319
loss evaluated:  3.21168
loss evaluated:  3.23559
loss evaluated:  3.13661
loss evaluated:  3.06134
loss evaluated:  2.94235
loss evaluated:  3.20458
loss evaluated:  3.19737
loss evaluated:  3.06679
